# 1. CML processing 
In this notebook cml are processed according to the following steps:
* calculate total loss
* simple quality control
* Wet&Dry Classification
* baseline computation 
* rain-induced attenuation
* rain rate estimate

In [16]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pycomlink as pycml
import pandas as pd
import poligrain as plg

## 1.1 CML PROCESSING & QUALITY CONTROL

In [17]:
def CML_preprocessing(ds_cmls):
    '''
    compute total loss and perform CML quality control
    '''
    # calculate total loss
    ds_cmls["tl"] = ds_cmls.tsl - ds_cmls.rsl

    # remove cmls with strong diurnal cycles
    keep = np.where(
        (
         (ds_cmls.tl.rolling(time=60 * 5, center=True).std() > 2).mean(dim="time") <= 0.1
        ).all(dim="sublink_id")
    )[0]
    ds_cmls = ds_cmls.isel(cml_id=keep)
    
    # remove cmls with very noisy periods
    keep = np.where(
        (
            (ds_cmls.tl.rolling(time=60, center=True).std() > 0.8).mean(dim="time") <= 0.35
        ).all(dim="sublink_id")
    )[0]
    ds_cmls = ds_cmls.isel(cml_id=keep)

    return ds_cmls

In [ ]:
def WetDryclassification(ds_cmls, rad):
    TimeSeq = pd.date_range(ds_cmls.time.values[0], ds_cmls.time.values[-1], freq="min")
    ds_cmls = ds_cmls.reindex({"time": TimeSeq})
    ds_cmls["wet"] = (("cml_id", "sublink_id", "time"), np.zeros(ds_cmls.rsl.shape, dtype="bool"))
    latitudes = np.array([rad.lat.values] * len(rad.lon.values)).transpose()
    longitudes = np.array([rad.lon.values] * len(rad.lat.values))
    da_intersect_weights = (
        plg.spatial.calc_sparse_intersect_weights_for_several_cmls(
            x1_line=ds_cmls.site_0_lon.values,
            y1_line=ds_cmls.site_0_lat.values,
            x2_line=ds_cmls.site_1_lon.values,
            y2_line=ds_cmls.site_1_lat.values,
            cml_id=ds_cmls.cml_id.values,
            x_grid=longitudes,
            y_grid=latitudes,
            grid_point_location="center",
        )
    )
    radar_along_cmls = (
        plg.spatial.get_grid_time_series_at_intersections(
            grid_data=rad.rainfall_amount,
            intersect_weights=da_intersect_weights,
        )
    )
    for ID in ds_cmls.cml_id.values:
        radAlong = radar_along_cmls.sel(cml_id=ID)
        # dry if radar along CML == 0
        dry_times = radAlong.time.where(radAlong == 0, drop=True).values
        mask = np.zeros(len(ds_cmls.time), dtype="bool")
        if len(dry_times) > 0:
            dry_intervals = np.array(
                [(t - np.timedelta64(15, "m"), t) for t in dry_times]
            )
            mask = np.any(
                [
                    (ds_cmls.time.values > start) & (ds_cmls.time.values <= end)
                    for start, end in dry_intervals
                ],
                axis=0,
            )
        ds_cmls["wet"].loc[{"cml_id": ID, "sublink_id": "channel1"}] = ~mask
        ds_cmls["wet"].loc[{"cml_id": ID, "sublink_id": "channel2"}] = ~mask   
    return ds_cmls 

In [18]:
def CML_processing(ds_cmls):
    '''
    cml processing
    '''
    # calculate wet periods
    roll_std_dev = ds_cmls.tl.rolling(time=60, center=True).std()
    threshold = 1.12 * roll_std_dev.quantile(0.8, dim="time")
    ds_cmls["wet_std"] = roll_std_dev > threshold
    
    # calculate baseline
    ds_cmls["baseline"] = pycml.processing.baseline.baseline_constant(
        trsl=ds_cmls.tl,
        wet=ds_cmls.wet_std,
        n_average_last_dry=5,
    )    

    # calculate wet antenna effect
    ds_cmls["A_obs"] = ds_cmls.tl - ds_cmls.baseline
    ds_cmls["A_obs"] = ds_cmls.A_obs.where(ds_cmls.A_obs >= 0, 0)
    ds_cmls["waa"] = pycml.processing.wet_antenna.waa_pastorek_2021_from_A_obs(
        A_obs=ds_cmls.A_obs,
        f_Hz=ds_cmls.frequency * 1e6,
        pol=ds_cmls.polarization.data,
        L_km=ds_cmls.length / 1000,
        A_max=6,
        zeta=0.7,  
        d=0.15, 
    )
    
    # calculate attenuation caused by rain and remove negative attenuation
    ds_cmls["A"] = ds_cmls.tl - ds_cmls.baseline - ds_cmls.waa
    ds_cmls["A"].data[ds_cmls.A < 0] = 0
    # derive rain rate via the k-R relation
    ds_cmls["R"] = pycml.processing.k_R_relation.calc_R_from_A(
        A=ds_cmls.A,
        L_km=ds_cmls.length.astype(float) / 1000,  
        f_GHz=ds_cmls.frequency / 1000,  
        pol=ds_cmls.polarization,
    )
    
    #calculate 1minute acc. precipitation  
    ds_cmls['R'] = ds_cmls['R']/60
    # leave only R 
    ds_cmls = ds_cmls.drop_vars(['wet_std', 'baseline', 'A_obs', 'rsl', 'tsl', 'tl', 'A', 'waa'])

    return ds_cmls


# how we use the measure .. mean of the cannels ?

## OpenMRG

In [13]:
#open dataset
ds_cmls = xr.open_dataset('data/andersson_2022_OpenMRG/openMRG_cml.nc')

In [14]:
# Resample to 1 minute temporal resolution
ds_cmls = ds_cmls.resample(time="1min").first(skipna=True)

In [ ]:
ds_cmls = CML_preprocessing(ds_cmls)
ds_cmls = CML_processing(ds_cmls) 

In [ ]:
ds_cmls.to_netcdf('CMLprocessed_OpenMRG.nc')

## OpenRainER

In [28]:
#open dataset
ds_cmls = xr.open_dataset('data/OpenRainER/CML_202101010000_202101312359.nc')

In [32]:
# correct CML lenght & polarization
ds_cmls['length'] = ds_cmls['length']*1000
ds_cmls['polarization'] = ds_cmls.polarization.fillna('vertical')

In [ ]:
ds_cmls = CML_preprocessing(ds_cmls)
ds_cmls = CML_processing(ds_cmls) 

In [ ]:
ds_cmls.to_netcdf('CMLprocessed_OpenRainER.nc')